<a href="https://colab.research.google.com/github/huhlim/cg2all/blob/main/cg2all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install cg2all package
!pip install -q git+http://github.com/huhlim/cg2all &> /dev/null
!pip install -q py3Dmol &> /dev/null

import torch
if torch.cuda.is_available():
    raise RuntimeError("Change the Runtime type to None.")

In [ ]:
#@title CG --> All-atom

# upload a PDB file
from google.colab import files

input_pdb = files.upload()
input_pdb = list(input_pdb)[0]

coarse_grained_model_type = "CalphaBasedModel" #@param ["CalphaBasedModel", "ResidueBasedModel", "Martini"]

# convert
import cg2all
import pathlib

output_pdb = pathlib.Path(input_pdb).stem + ".all.pdb"
_ = cg2all.convert_cg2all(input_pdb, output_pdb, model_type=coarse_grained_model_type)
print(f"Converted {input_pdb} in {coarse_grained_model_type} to {output_pdb}")

# display output
import py3Dmol

with open(output_pdb, "r") as fp:
    output = fp.read()
    output = output.replace("OT1", "O  ")
    output = output.replace("OT2", "OXT")

view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
view.addModel(output, 'pdb')
view.setStyle({'cartoon': {'color':'spectrum'}})
  
BB = ['C','O','N']
view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                    {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.2}})
view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                    {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.2}})
view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                    {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.2}})  

view.zoomTo()
view.show()

In [ ]:
#@title Download the converted file
files.download(output_pdb)